<a href="https://colab.research.google.com/github/sugartastic81/CVdepthAI_OAK-D_TensionTerminator/blob/main/trainTTyolo5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tension Terminator YOLO5 Transfer Learning

## Set global variables

In [ ]:
# Set Variables
BASE_MODEL = "yolov5m"
TRAIN_EPOCHS = 70
TRAIN_SIZE = 320

TT_PROJECTNAME = "TensionTerminator"

## Mount Google Drive to backup files

In [ ]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

google_drive_target_folder = '/content/drive/My Drive/'+TT_PROJECTNAME+'/'
# Check if the target folder exists in Google Drive, create if it doesn't
if not os.path.exists(google_drive_target_folder):
    os.makedirs(google_drive_target_folder)
    print(f"Created folder: {google_drive_target_folder}")
print("Google Drive Target Folder: " + google_drive_target_folder)

# Clone YoloV5 GIT

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install
%cd ..

import torch
from yolov5 import utils

## Install Dependencies

In [ ]:
!pip install roboflow

In [ ]:
import os
import random
import yaml
import shutil
from google.colab import userdata, files
from datetime import datetime
from roboflow import Roboflow

### Load the data from roboflow

In [ ]:
secret_key = userdata.get('rb-key')

rf = Roboflow(api_key=secret_key)
project = rf.workspace("juzaworkspace-wzpkn").project("tensionterminatordetectionv2")
dataset = project.version(5).download("yolov5")

DATASET_DIR = dataset.location
DATA_YAML = DATASET_DIR + "/data.yaml"
print("\n\info")
print("Data saved to: " + DATASET_DIR)
print("Data YAML: " + DATA_YAML)

In [ ]:
# corrects the folders in the data.yaml

# Path to the YAML file
yaml_file_path = DATASET_DIR + 'data.yaml'

# Read the YAML file
with open(DATA_YAML) as file:
    data = yaml.safe_load(file)

# Update the paths
data['train'] = DATASET_DIR + '/train/images'
data['val'] = DATASET_DIR + '/valid/images'
data['test'] = DATASET_DIR + '/test/images'

# Write the updated data back to the YAML file
with open(DATA_YAML, 'w') as file:
    yaml.dump(data, file)

print("Updated data.yaml successfully.")

### Training of the model

In [ ]:
model_training_command = "python ./train.py --img " + str(TRAIN_SIZE) + " --batch 8 --epochs " + str(TRAIN_EPOCHS) + " --data " + DATA_YAML + " --weights " + BASE_MODEL + ".pt --cache"
print(model_training_command)

In [ ]:
%cd /content/yolov5
!{model_training_command}
#!python ./train.py --img 640 --batch 8 --epochs 100 --data ./OverheadPersonDetection-3/data.yaml --weights yolov5m.pt --cache

### Download the trained weights file

In [ ]:
# Get latest run dir
base_dir = '/content/runs/detect'
latest_run_dir = max([os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))], key=os.path.getmtime)
print(latest_run_dir)

# modify if there is an error
#latest_run_dir = "/content/runs/detect/train3"

# Define the base directory where the runs are stored
TRAINED_WEIGHTS = latest_run_dir + "/weights/" + "best.pt"

# Construct the path to the best weights file
print("Original trained weights path: ", TRAINED_WEIGHTS)

In [ ]:
# Get the current timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Construct the new file name with timestamp
new_file_name = f"TTmodel2_{BASE_MODEL}_img{TRAIN_SIZE}_{timestamp}.pt"
new_file_path = os.path.join(latest_run_dir, 'weights', new_file_name)

# Copy the file with timestamp in the same directory
shutil.copy2(TRAINED_WEIGHTS, new_file_path)
print("New file path with timestamp: ", new_file_path)

# Make sure the target folder exists
if not os.path.exists(google_drive_target_folder):
    os.makedirs(google_drive_target_folder)

# Construct the target file path in Google Drive
google_drive_target_path = os.path.join(google_drive_target_folder, new_file_name)

# Copy the file to Google Drive
shutil.copy2(new_file_path, google_drive_target_path)
print("Model file copied to Google Drive: ", google_drive_target_path)